# Проверка работы Word2Vec

### Настраиваем корневую папку

In [1]:
import first_script

### Импорт нужных библиотек

In [2]:
from model import GPT
from tokenizer import FastTextTokenizer
import os

### Загружаем текст на котором будем обучать модель

In [3]:
# text = "Мама мыла раму, пока кошка ходила по улице, а папа ел хлеб. Дети гуляли по улице!"
text = ""
with open('../data/voina_i_mir.txt', 'r') as f:
    text = f.read()

### Загружаем предобученый токенизатор

In [4]:
tokenizer = FastTextTokenizer(n_gram=3)
tokenizer.train(text)

### Превращаем текст в индексы

In [5]:
text_idxs = tokenizer.encode(text, add_special_tokens=True)

In [6]:
print(tokenizer.decode(text_idxs[:100]))
print(tokenizer.decode(text_idxs[:100], add_special_tokens=True))

['Лев', 'Николаевич', 'Толстой', 'Война', 'и', 'мир', '.', 'Том', '1', 'Лев', 'Николаевич', 'Толстой', 'ВОЙНА', 'И', 'МИР']
['[BOP]', '[BOS]', 'Лев', 'Николаевич', 'Толстой', '[EOS]', '[EOP]', '[BOP]', '[EOP]', '[BOP]', '[BOS]', 'Война', 'и', 'мир', '.', '[EOS]', '[BOS]', 'Том', '1', '[EOS]', '[EOP]', '[BOP]', '[EOP]', '[BOP]', '[EOP]', '[BOP]', '[EOP]', '[BOP]', '[EOP]', '[BOP]', '[EOP]', '[BOP]', '[EOP]', '[BOP]', '[EOP]', '[BOP]', '[BOS]', 'Лев', 'Николаевич', 'Толстой', '[EOS]', '[EOP]', '[BOP]', '[EOP]', '[BOP]', '[BOS]', 'ВОЙНА', 'И', 'МИР', '[EOS]', '[EOP]', '[BOP]']


### Параметры модели

In [7]:
embed_dim = 128
seq_size = 20
hidden_size = embed_dim * 2
num_layers = 3
num_heads = 32
device = 'cuda'

### Параметры обучения

In [8]:
num_epochs = 5
batch_size = 480
num_workers = os.cpu_count()

### Загружаем модель и обучаем

In [9]:
model = GPT(len(tokenizer.vocab), embed_dim, hidden_size, num_heads, num_layers, seq_size, device=device)

In [10]:
res = model.fit(text_idxs, batch_size, num_epochs, num_workers)

Epoch [1/5, time: 18.234 minutes]:
                            Train Loss 59.557521, Train Perplexity 73367349803206840820957184.000
                            Test Loss 28.714711, Test Perplexity 2955565031038.195
Epoch [2/5, time: 36.550 minutes]:
                            Train Loss 14.201656, Train Perplexity 1471298.811
                            Test Loss 19.421203, Test Perplexity 271970277.917
Epoch [3/5, time: 54.517 minutes]:
                            Train Loss 7.882394, Train Perplexity 2650.209
                            Test Loss 17.218738, Test Perplexity 30060957.749
Epoch [4/5, time: 71.775 minutes]:
                            Train Loss 5.553613, Train Perplexity 258.169
                            Test Loss 16.787079, Test Perplexity 19522521.950
Epoch [5/5, time: 89.378 minutes]:
                            Train Loss 4.480645, Train Perplexity 88.292
                            Test Loss 16.932094, Test Perplexity 22569139.569


### Визуализируем метрики обучения

In [11]:
model.plot_metrics(*res)

### Генерируем текст

In [17]:
test = "Привет, как дела?"
test_idxs = tokenizer.encode(test, add_special_tokens=True)
test_pred_idxs = model.generate_text(test_idxs, text_size=50, top_k=50)
test_pred = tokenizer.decode(test_pred_idxs, join=True)
print(test_pred)

Привет, как дела? – говорила говорила говорила


In [18]:
test = "Привет, как дела?"
test_idxs = tokenizer.encode(test, add_special_tokens=False)
test_pred_idxs = model.generate_text(test_idxs, text_size=50, top_k=50)
test_pred = tokenizer.decode(test_pred_idxs, join=True)
print(test_pred)

Привет, как дела? – сказала она хорошо. – Я мать.


In [36]:
test = "Он лежал у дуба, а "
test_idxs = tokenizer.encode(test, add_special_tokens=True)
test_pred_idxs = model.generate_text(test_idxs, text_size=50)
test_pred = tokenizer.decode(test_pred_idxs, join=True)
print(test_pred)

Он лежал у дуба, а Он сная? Он зная? Я? Он? Он?


In [39]:
test = "Андрей смотрел"
test_idxs = tokenizer.encode(test, add_special_tokens=False)
test_pred_idxs = model.generate_text(test_idxs, text_size=50)
test_pred = tokenizer.decode(test_pred_idxs, join=True)
print(test_pred)

Андрей смотрел на пугадачив улыбнулся. Анна Милый лице


Чуть лучше, чем у обычной модели. Для полтора часа обучения, такой небольшой выборке и моем железе вполне даже хорошо